# I. Select games and generate random sample

In [1]:
from Prob_game.sample_generation import *

In [2]:
n_sample_ = 500

In [3]:
# Skew Norm
skewnorm = Sample_generation(count = 99, probs = "skewnorm", a = 10, change_bomb = "T", n_play = 50 )
sknewnorm_df = skewnorm.gen_sample(flag = 1, n_sample = n_sample_)

# Random Skew
random_skew = Sample_generation(count= 99, probs = "random_skew", peak = 80, smooth = "True", change_bomb = "T", n_play = 50 )
random_skew_df = random_skew.gen_sample(flag = 2, n_sample = n_sample_)

# Increasing
increasing = Sample_generation(count= 99, probs = "increasing", change_bomb = "T", n_play = 50 )
increasing_df = increasing.gen_sample(flag = 3, n_sample = n_sample_)

In [4]:
final_df = pd.DataFrame()

final_df = final_df.append(sknewnorm_df, ignore_index = True)
final_df = final_df.append(random_skew_df, ignore_index = True)
final_df = final_df.append(increasing_df, ignore_index = True)

# II. Train Classification NN

In [5]:
from Prob_game.classification import *

Using TensorFlow backend.


In [6]:
class_nn = Classification_type(final_df)

In [7]:
class_nn.classification_nn()

C:\Users\zq113rm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Model is trained


In [8]:
import numpy as np

# III. Evolutionary optimization

In [9]:
from Prob_game.nn_evolution import *

In [11]:
skewnorm_evolution = Nn_evolution(count = 99, probs = "skewnorm", change_bomb = "T", n_play = 50, n_pop = 30,  n_gen = 40 , elitism = 0.6 , loser_percent = 0)
skewnorm_evolution.evolve(a = 10)
skewnorm_evolution_best = skewnorm_evolution.select_best_nn()

100%|██████████| 40/40 [02:54<00:00,  4.14s/it]


In [12]:
random_skew_evolution = Nn_evolution(count = 99, probs = "random_skew", change_bomb = "T", n_play = 50, n_pop = 30,  n_gen = 40 , elitism = 0.6 , loser_percent = 0)
random_skew_evolution.evolve(peak = 80, smooth = "True")
random_skew_evolution_best = random_skew_evolution.select_best_nn()

100%|██████████| 40/40 [00:27<00:00,  1.13it/s]


In [13]:
increasing_evolution = Nn_evolution(count = 99, probs = "increasing", change_bomb = "T", n_play = 50, n_pop = 30,  n_gen = 40 , elitism = 0.6 , loser_percent = 0)
increasing_evolution.evolve()
increasing_evolution_best = increasing_evolution.select_best_nn()

100%|██████████| 40/40 [00:29<00:00,  1.28it/s]


# IV. Testing

In [14]:
import random

In [15]:
from Prob_game.risky_bomb import *

In [18]:
count = 99
r_model = random.randint(1,3)

if r_model == 1: 
    base = Risky_bomb(count = count, probs = "skewnorm", a = 10)
elif r_model == 2:
    base = Risky_bomb(count = count, probs = "random_skew", peak = 80, smooth = "True")
else:  
    base = Risky_bomb(count = count, probs = "increasing")

In [19]:
r_model

1

In [ ]:

# Play random rounds in order to produce more accurate classification
guess_in = []
p = []



learn_ = 5
for j in range(0,learn_):
    guess_in.append(np.random.randint(1, count))
    p.append(base.payment(guess = guess_in[j], change_bomb = self.change_bomb))

In [ ]:
guess_in = [np.random.randint(1, self.count)]
p = [self.base.payment(guess = guess_in[0] , change_bomb = self.change_bomb)]
opt_ = 25
learn_ = int(self.n_play / 3)
for j in range(1,learn_):
    guess_in.append(np.random.randint(1, self.count))
    p.append(self.base.payment(guess = guess_in[j], change_bomb = self.change_bomb))

for i in range(learn_, self.n_play - learn_):
    p_h = np.array(p)
    g_h = np.array(guess_in)
    try: 
        opt_ = int(np.mean(list(p_h[np.where(p_h > 0)])))
    except ValueError:
        opt_ = opt_
    guess_in.append(opt_)
    p.append(self.base.payment(guess = guess_in[i], change_bomb = self.change_bomb))
out_ = pd.DataFrame({"payment":p, "guesses":guess_in})
return(out_)

In [ ]:
        df_out = pd.DataFrame(columns = ["Sample_ID",
                                         "seq", 
                                         "flag",
                                         "payment",
                                         "guesses",
                                         "bomb",
                                         "mean_guess",
                                         "std_guess",
                                         "mean_p",
                                         "std_p",
                                         "last_3_p_mean"])
        for j in range(0, n_sample - 1):
            
             # Initiate values
            seq_ = []
            flag_ = []
            guess_in = []
            p = []
            bomb = []
            mean_guess = []
            std_guess = []
            mean_p = []
            std_p = []
            last_3_p_mean= []

            for i in range(0, self.n_play):  
                seq_.append(i)
                flag_.append(flag)
                guess_in.append(np.random.randint(0,self.count))
                p.append(self.base.payment(guess = guess_in[i], change_bomb = self.change_bomb))
                bomb.append(self.base.bomb)
                mean_guess.append(np.mean(guess_in))
                std_guess.append(np.std(guess_in))
                mean_p.append(np.mean(p))
                std_p.append(np.std(p))
                if len(p) < 3:
                    c = 0
                else: 
                    c = len(p) - 3
                last_3_p_mean.append(np.mean(p[c:]))
                
            out_ = pd.DataFrame({"Sample_ID": j,
                                 "seq": seq_,
                                 "flag": flag_,
                                 "payment": p,
                                 "guesses": guess_in, 
                                 "bomb":bomb,
                                 "mean_guess": mean_guess, 
                                 "std_guess": std_guess,
                                 "mean_p": mean_p, 
                                 "std_p": std_p,
                                 "last_3_p_mean": last_3_p_mean})
            
            df_out = df_out.append(out_ , ignore_index = True)
        
        self.gen_sample_df = df_out
        return(df_out)


In [16]:
# Select model randomly
r_model = random.randint(1,3)

if r_model == 1: 
    opt_weight = skewnorm_evolution_best
elif r_model == 2:
    opt_weight = random_skew_evolution_best 
else:  
    opt_weight = increasing_evolution_best 